In [1]:
import torch
import torch.nn as nn
import snntorch as snn
import numpy as np
from snntorch import import_nirtorch
import nir


In [2]:
graph = nir.read("scnn_mnist.nir")
graph.nodes.keys()

dict_keys(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9', 'input', 'output'])

In [3]:
net = import_nirtorch.from_nir(graph)

replace rnn subgraph with nirgraph


In [4]:
net

GraphExecutor(
  (0): Conv2d(34, 34, kernel_size=(34, 34), stride=(2, 2), padding=(1, 1))
  (1): Leaky()
  (10): Leaky()
  (11): Linear(in_features=256, out_features=10, bias=True)
  (12): Leaky()
  (2): Conv2d(16, 16, kernel_size=(16, 16), stride=(1, 1), padding=(1, 1))
  (3): Leaky()
  (4): LPPool2d(norm_type=1, kernel_size=(2, 2), stride=(2, 2), ceil_mode=False)
  (5): Conv2d(8, 8, kernel_size=(8, 8), stride=(1, 1), padding=(1, 1))
  (6): Leaky()
  (7): LPPool2d(norm_type=1, kernel_size=(2, 2), stride=(2, 2), ceil_mode=False)
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=128, out_features=256, bias=True)
  (input): Identity()
  (output): Identity()
)

In [5]:
numbers = torch.from_numpy(np.load("val_numbers.npy"))
numbers.shape

torch.Size([300, 10, 2, 34, 34])

In [6]:
modules = list(net.children())

In [8]:
out = [] 
# modules[1].mem = modules[1].init_leaky()
for t in numbers.float():
  t = modules[0](t)
  spk = modules[1](t)
  out.append(spk)
out = torch.stack(out).detach()
out.shape

torch.Size([300, 10, 16, 16, 16])

In [15]:
np.save("snnTorch_activity.npy", out.numpy())